<a href="https://colab.research.google.com/github/ashikshafi08/Learning_Tensorflow/blob/main/TensorFlow/Emotion_Detection_AI_Crowd_Clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install aicrowd-cli
API_KEY = '' 
!aicrowd login --api-key $API_KEY

# Downloading the Dataset
!mkdir data
!aicrowd dataset download --challenge emotion-detection -j 3 -o data

     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 61kB 5.4MB/s 
     |████████████████████████████████| 215kB 35.9MB/s 
     |████████████████████████████████| 61kB 6.8MB/s 
     |████████████████████████████████| 174kB 39.6MB/s 
     |████████████████████████████████| 81kB 8.7MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 71kB 8.8MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
API Key valid
Saved API Key successfully!
test.csv:   0% 0.00/642k [00:00<?, ?B/s]
train.csv:   

In [2]:
# Importing all the packages we need 
import tensorflow as tf 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# Importing the data 

train_data = pd.read_csv('data/train.csv')
val_data = pd.read_csv('data/val.csv')
test_data = pd.read_csv('data/test.csv')

# Printing out all shapes of our data 
print(f'Shape of the train data: {train_data.shape}')
print(f'Shape of the validation data: {val_data.shape}')
print(f'Shape of the test data: {test_data.shape}')

Shape of the train data: (31255, 2)
Shape of the validation data: (3473, 2)
Shape of the test data: (8682, 2)


In [4]:
# Shuffling our train data 
train_data_shuffled = train_data.sample(frac = 1 , random_state = 42)
train_data_shuffled.head() , train_data_shuffled.shape

(                                                    text  label
 13976                                           Yaaaay!!      0
 3256   says the insecure idiot who just had to dig th...      0
 24501  Game still "fun" if you don't tryhard and play...      0
 22477  Connie Drake learned to fight smarter, not har...      0
 16658               And she jumped into sewage. Yeh, no.      1, (31255, 2))

In [5]:
# Splitting sentences and labels
train_sentences = train_data_shuffled['text'].to_numpy()
train_labels = train_data_shuffled['label'].to_numpy()

val_sentences = val_data['text'].to_numpy()
val_labels = val_data['label'].to_numpy()

test_sentences = test_data['text'].to_numpy()
test_labels = test_data['label'].to_numpy()


# Checking the shapes 
print(f'Shape of the train sentences: {train_sentences.shape}')
print(f'Shape of the validation sentences: {val_sentences.shape}')
print(f'Shape of the train labels: {train_labels.shape}')
print(f'Shape of the validation labels: {val_labels.shape}')

Shape of the train sentences: (31255,)
Shape of the validation sentences: (3473,)
Shape of the train labels: (31255,)
Shape of the validation labels: (3473,)


In [58]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor = 'val_accuracy' , 
    factor = 0.2 , 
    patience = 0 , 
    min_lr = 0.0001
)

In [55]:
# Creating a Keras Layer (Using Transfer Learning)

sentence_encoder_layer = hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4' , 
                                  input_shape = [], 
                                  dtype = tf.string , 
                                  trainable = False , 
                                  name = 'USE_layer')

In [74]:
# Building a model
from tensorflow.keras import layers

model = tf.keras.Sequential([
  sentence_encoder_layer,
  layers.Dense(128 , activation = 'relu'),
  layers.Dense(1 , activation = 'sigmoid')

] , name = 'USE_model')

# Summary of the mdoel 
model.summary()

Model: "USE_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
USE_layer (KerasLayer)       (None, 512)               256797824 
_________________________________________________________________
dense_25 (Dense)             (None, 128)               65664     
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 129       
Total params: 256,863,617
Trainable params: 65,793
Non-trainable params: 256,797,824
_________________________________________________________________


In [75]:
# Compiling a model 
model.compile(loss = tf.keras.losses.BinaryCrossentropy() , 
                   optimizer = tf.keras.optimizers.Adam() , 
                   metrics = ['accuracy'])

# Fit the model 
history = model.fit(train_sentences , train_labels , 
               validation_data = (val_sentences , val_labels) , 
               epochs = 15)

Epoch 1/15
977/977 [==============================] - 13s 12ms/step - loss: 0.4129 - accuracy: 0.8133 - val_loss: 0.3815 - val_accuracy: 0.8226
Epoch 2/15
977/977 [==============================] - 11s 12ms/step - loss: 0.3796 - accuracy: 0.8275 - val_loss: 0.3776 - val_accuracy: 0.8244
Epoch 3/15
977/977 [==============================] - 11s 12ms/step - loss: 0.3654 - accuracy: 0.8348 - val_loss: 0.3809 - val_accuracy: 0.8166
Epoch 4/15
977/977 [==============================] - 11s 12ms/step - loss: 0.3490 - accuracy: 0.8458 - val_loss: 0.3764 - val_accuracy: 0.8200
Epoch 5/15
977/977 [==============================] - 12s 12ms/step - loss: 0.3308 - accuracy: 0.8545 - val_loss: 0.3795 - val_accuracy: 0.8241
Epoch 6/15
977/977 [==============================] - 12s 12ms/step - loss: 0.3135 - accuracy: 0.8646 - val_loss: 0.3839 - val_accuracy: 0.8209
Epoch 7/15
977/977 [==============================] - 12s 12ms/step - loss: 0.2949 - accuracy: 0.8746 - val_loss: 0.3944 - val_accuracy:

In [68]:
# Getting the prediction probs
pred_probs = model.predict(test_sentences)
pred_probs[:10]

# Converting into labels 
model_preds = tf.cast(tf.squeeze(tf.round(pred_probs)) , tf.int32)
model_preds[:10]

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([1, 1, 0, 0, 1, 0, 0, 1, 0, 0], dtype=int32)>

In [69]:
# Dropping the label column
test_data.drop('label' , inplace = True , axis = 1)

# Assigning the predictions
test_data['label'] = model_preds
test_data.head()


,text,label
0,I was already over the edge with Cassie Zamora...,1
1,I think you're right. She has oodles of cash a...,1
2,Haha I love this. I used to give mine phone bo...,0
3,Probably out of desperation as they going no a...,0
4,Sorry !! You’re real good at that!!,1


In [35]:
# Saving the model 
model.save('use_tf_model')

INFO:tensorflow:Assets written to: use_tf_model/assets


INFO:tensorflow:Assets written to: use_tf_model/assets


In [20]:
# Loading the model as inference 
inf_model = tf.keras.models.load_model('use_tf_model/')
inf_model.summary()

Model: "USE_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
USE_layer (KerasLayer)       (None, 512)               256797824 
_________________________________________________________________
dense_3 (Dense)              (None, 64)                32832     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [21]:
# Evaluating on the validation data
inf_model.evaluate(val_sentences , val_labels)

109/109 [==============================] - 1s 9ms/step - loss: 0.3750 - accuracy: 0.8232


[0.3749535381793976, 0.8232076168060303]

In [70]:
import os
#!mkdir assets
test_data.to_csv(os.path.join("assets", "submission.csv"), index=False)

In [71]:
!aicrowd notebook submit -c emotion-detection -a assets --no-verify

Using notebook: /content/drive/MyDrive/Colab Notebooks/Inference_Notebook.ipynb for submission...
Removing existing files from submission directory...
Scrubbing API keys from the notebook...
submission.zip ━━━━━━━━━━━━━━━━━━ 100.0% • 293.9/292.2 KB • 957.5 kB/s • 0:00:00
                                                  ╭─────────────────────────╮                                                  
                                                  │ Successfully submitted! │                                                  
                                                  ╰─────────────────────────╯                                                  
                                                        Important links                                                        
┌──────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│  This submission │ https://www.aicrowd.com/challenges/ai-blitz-9/problems/emotion-dete